In [1]:
!pip install -U datasets tqdm pandas numpy uuid datetime requests torch sentence-transformers groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 721.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 35.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.2 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [3]:
from huggingface_hub import login
login(new_session=False,token=HF_TOKEN)

## Imports

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from sentence_transformers import SentenceTransformer
import torch
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import uuid
import json
import datetime
import re
try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    print("Groq library not found. Install with 'pip install groq' if you plan to use Groq API.")
    GROQ_AVAILABLE = False



ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

## Configuration

In [12]:
DATASET_NAME = "hamzabouajila/tunisian-derja-unified-raw-corpus"
OUTPUT_FILE = "tunisian_msa_parallel_corpus.jsonl"
MAX_ATTEMPTS = 3
SCORE_THRESHOLD = 0.7 # Composite score threshold for acceptance
USE_GROQ = True # Set to False to use local LLM function
LOCAL_LLM_NAME = "your_local_model_name_or_path" # Specify if not using Groq
GROQ_MODEL = "llama3-70b-8192" # Example Groq model
# GROQ_MODEL = "mixtral-8x7b-32768" # Another option
BATCH_SIZE = 10 # Process sentences in batches for efficiency (adjust as needed)
SPLIT_RATIO = {"train": 0.8, "validation": 0.1, "test": 0.1} # Approximate split ratio
# --- Initialize Groq Client (if using) ---
if USE_GROQ and GROQ_AVAILABLE:
    try:
        groq_client = Groq(api_key=GROQ_API_KEY)
        if not groq_client.api_key:
            raise ValueError("GROQ_API_KEY environment variable not set.")
    except Exception as e:
        print(f"Error initializing Groq client: {e}")
        USE_GROQ = False
elif USE_GROQ and not GROQ_AVAILABLE:
    USE_GROQ = False



## Preprocessing Functions

### Translation

In [33]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-4B-Instruct-2507", trust_remote_code=True)
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [24]:

translation_pipe =  pipeline("text-generation", model="Qwen/Qwen3-4B-Thinking-2507")

def translate_tn_to_msa_local(tn_sentence):
    """Placeholder for translating using a local LLM."""
    # Implement your local LLM translation logic here.
    # Example using transformers pipeline:
    try:
        messages = [
            {"role": "user", "content": f"ترجمة الجملة التونسية إلى العربية الفصحى: {tn_sentence}"},
                ]
        result = translation_pipe(messages, temperature=0.5)
        return result[0]['generated_text'][-1]["content"].split("\n\n")[-1].split("الترجمة هي:")[-1].strip().replace("**","")
    # .split("MSA Translation:")[-1].strip()
    except Exception as e:
         print(f"Error translating '{tn_sentence}' with local LLM: {e}")
         return None
    #Example using llama-cpp-python (requires setup):
    # from llama_cpp import Llama
    # llm = Llama(model_path="path/to/your/gguf/model")
    # prompt = f"Translate the following Tunisian Arabic text to Modern Standard Arabic:\n\n{tn_sentence}\n\nMSA Translation:"
    # output = llm(prompt, max_tokens=512, stop=["\n"], echo=True)
    # return output['choices'][0]['text'].split("MSA Translation:")[-1].strip()

    # Placeholder return
    # print(f"Local LLM translation not implemented for: {tn_sentence}")
    # return f"[LOCAL_LLM_TRANSLATION_PLACEHOLDER] {tn_sentence}" # Indicate it's a placeholder


# --- Translation Functions ---
def translate_tn_to_msa_groq(tn_sentence):
    """Translate Tunisian Arabic to MSA using Groq API."""
    if not USE_GROQ or not GROQ_AVAILABLE:
        raise ValueError("Groq API not configured or available.")
    try:
        prompt = f"Translate the following Tunisian Arabic text to Modern Standard Arabic:\n\n{tn_sentence}\n\nMSA Translation:"
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=GROQ_MODEL,
            temperature=0.2, # Lower temperature for more deterministic output
            max_tokens=512, # Adjust based on expected sentence length
            top_p=1,
            stop=None,
            stream=False,
        )
        msa_translation = chat_completion.choices[0].message.content.strip()
        return msa_translation
    except Exception as e:
        print(f"Error translating '{tn_sentence}' with Groq: {e}")
        return None




def translate_tn_to_msa(tn_sentence):
    """Wrapper to select translation method."""
    if USE_GROQ and GROQ_AVAILABLE:
        return translate_tn_to_msa_groq(tn_sentence)
    else:
        return translate_tn_to_msa_local(tn_sentence)


ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [18]:
translate_tn_to_msa_local("شنوة أحوالك ؟")

'أحوالك ؟ شنوة أحوال:: URL'

### preprocess_sentence

In [ ]:
def preprocess_sentence(text):
    """Basic preprocessing for Tunisian Arabic text."""
    if not isinstance(text, str):
        return None
    # Basic cleaning: strip whitespace, remove extra newlines
    text = text.strip()
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespaces with single space
    # Add more sophisticated cleaning if needed (e.g., normalize specific characters)
    # Filter out very short sentences or sentences with only numbers/punctuation?
    if len(text) < 5: # Example filter
        return None
    return text

### is_valid_tunisian_arabic

In [ ]:
dialect_identifier_pipeline = pipeline('text-classification', model='Ammar-alhaj-ali/arabic-MARBERT-dialect-identification-city')


def is_valid_tunisian_arabic(text):
    """
    Determines if the input text is Tunisian Arabic using a pre-trained model.
    
    Args:
        text (str): The text to classify.
        
    Returns:
        bool: True if the text is identified as Tunisian Arabic, False otherwise.
    """
    if not text or not isinstance(text, str):
        return False
        
    try:
        predictions = dialect_identifier_pipeline([text])
        if not predictions or 'label' not in predictions[0]:
            return False
        predictions = predictions[0]['label']
        return predictions in ["Tunis","Sfax"]
        
    except Exception as e:
        # Handle any potential errors during prediction
        print(f"Error in dialect identification: {e}")
        return False

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
is_valid_tunisian_arabic( "أنا راجل تونسي")

predincting أنا راجل تونسي


True

### calculate_semantic_similarity

In [ ]:

# Load the pre-trained Arabic embedding model once, outside the function, for efficiency.
# 'UBC-NLP/MARBERT' or 'UBC-NLP/AraT5-base' are strong choices for Arabic.
# As per the search results, newer models like the Arabic Matryoshka Embedding Models 
# or OMER NACAR models are also excellent options.
semantic_similarity_model = SentenceTransformer('aubmindlab/bert-base-arabertv02') # Replace with your chosen model



def calculate_semantic_similarity(tn_sentence, msa_candidate, model=semantic_similarity_model):
    """
    Calculate the cosine similarity between the embeddings of a Tunisian Arabic sentence 
    and an MSA candidate sentence.

    Args:
        tn_sentence (str): The Tunisian Arabic sentence.
        msa_candidate (str): The Modern Standard Arabic sentence.
        model: A pre-trained SentenceTransformer model.

    Returns:
        float: The cosine similarity score between 0.0 (no similarity) and 1.0 (identical).
    """
    if not tn_sentence or not msa_candidate or not isinstance(tn_sentence, str) or not isinstance(msa_candidate, str):
        return 0.0

    try:
        # Encode both sentences into dense vector embeddings.
        embeddings = model.encode([tn_sentence, msa_candidate])
        
        # Extract the embeddings for each sentence.
        emb1 = embeddings[0]  # Embedding for Tunisian sentence
        emb2 = embeddings[1]  # Embedding for MSA sentence

        # Calculate cosine similarity.
        cos_sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        
        # Ensure the result is a float and clip to valid range due to potential floating-point errors.
        return float(np.clip(cos_sim, 0.0, 1.0))

    except Exception as e:
        print(f"Error calculating semantic similarity: {e}")
        return 0.0


2025-08-30 10:56:27.292583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756551387.321734    1297 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756551387.330906    1297 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
calculate_semantic_similarity("أنا راجل تونسي", "أنا رجل تونسي",semantic_similarity_model)

### calculate_backtranslation_score

In [ ]:
def calculate_backtranslation_score(original_tn, backtranslated_tn, model=None):
    """Calculate similarity between original TN and back-translated TN."""
    # Requires a TN -> MSA -> TN pipeline or a dedicated TN-MSA-TN model
    # Example similarity calculation (e.g., using embedding similarity again)
    try:
        similarity = calculate_semantic_similarity(original_tn, backtranslated_tn, model=model) # Reuse semantic sim function
        return similarity
    except Exception as e:
        print(f"Error calculating backtranslation score: {e}")
        return 0.0


In [ ]:
calculate_backtranslation_score("أنا راجل تونسي", "أنا رجل تونسي",semantic_similarity_model)

### calculate_lm_fluency

In [ ]:
# Load the tokenizer and model once, outside the function, for efficiency.
# 'aubmindlab/aragpt2-base' is a good choice for Modern Standard Arabic.
# Other models like 'marefa-nlp/ajeeb-gpt2-large-ar' or 'CAMeL-Lab/bert-base-arabic' 
# (adapted for sequence scoring) are also available.

lm_fluency_tokenizer = AutoTokenizer.from_pretrained('aubmindlab/aragpt2-large', trust_remote_code=True)
lm_fluentcy_model = AutoModelForCausalLM.from_pretrained('aubmindlab/aragpt2-large', trust_remote_code=True)

def calculate_lm_fluency(msa_candidate, model=lm_fluentcy_model, tokenizer=lm_fluency_tokenizer):
    """
    Calculate fluency score for an MSA sentence using a pre-trained Arabic Language Model.
    The score is the negative average log-likelihood. Lower perplexity (higher log-likelihood) 
    indicates higher fluency.
    """
    if not msa_candidate or not isinstance(msa_candidate, str):
        return float('-inf')

    try:
        # Tokenize the input text.
        inputs = tokenizer(msa_candidate, return_tensors="pt", truncation=True, max_length=512)
        
        with torch.no_grad():  # Disable gradient calculation for inference.
            # Get the model's outputs. Pass the input_ids as labels to compute the loss.
            outputs = model(**inputs, labels=inputs["input_ids"])
            # The loss is the cross-entropy loss, which is the average negative log-likelihood.
            avg_log_likelihood = -outputs.loss.item() # Use .item() to get a Python float

        return avg_log_likelihood

    except Exception as e:
        print(f"Error calculating LM fluency: {e}")
        return float('-inf')

In [ ]:
calculate_lm_fluency("أنا رجل تونسي",lm_fluentcy_model,lm_fluency_tokenizer)

-8.509749412536621

### calculate_ensemble_agreement

In [ ]:


def calculate_ensemble_agreement(msa_candidates, model=None):
    """Calculate agreement among multiple MSA candidates."""
    # Example: Generate N candidates (e.g., with different sampling params)
    # Calculate pairwise similarities and average them
    if len(msa_candidates) < 2:
        return 1.0 # Perfect agreement if only one
    try:
        similarities = []
        for i in range(len(msa_candidates)):
            for j in range(i+1, len(msa_candidates)):
                sim = calculate_semantic_similarity(msa_candidates[i], msa_candidates[j], model=model)
                similarities.append(sim)
        avg_agreement = np.mean(similarities) if similarities else 0.0
        return float(avg_agreement)
    except Exception as e:
        print(f"Error calculating ensemble agreement: {e}")
        return 0.0

In [ ]:
calculate_ensemble_agreement(["أنا رجل تونسي", "أنا رجل من تونس","أنا تونسي"],semantic_similarity_model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.8734367688496908

### calculate_composite_score

In [ ]:

def calculate_composite_score(scores_dict):
    """Calculate the final composite score based on individual scores."""
    # Define weights for each component (you can adjust these)
    w_semantic = 0.3
    w_fluency = 0.2
    w_backtrans = 0.3
    w_ensemble = 0.2

    # Normalize scores if needed (especially logprob)
    semantic_score = scores_dict.get('semantic_similarity', 0.0)
    fluency_score = scores_dict.get('lm_logprob', float('-inf'))
    backtrans_score = scores_dict.get('backtranslation_score', 0.0)
    ensemble_score = scores_dict.get('ensemble_agreement', 0.0)

    # Simple normalization for logprob (example - adjust based on your model's range)
    # Assuming logprob is negative, higher (closer to 0) is better
    normalized_fluency = 1.0 / (1.0 + np.exp(-fluency_score)) if fluency_score != float('-inf') else 0.0

    composite = (
        w_semantic * semantic_score +
        w_fluency * normalized_fluency +
        w_backtrans * backtrans_score +
        w_ensemble * ensemble_score
    )
    return composite

In [ ]:
calculate_composite_score({"semantic_similarity": 0.8, "lm_logprob": -1.5, "backtranslation_score": 0.75, "ensemble_agreement": 0.9})

0.6814851047612712

In [ ]:

# --- Main Pipeline ---
def generate_parallel_corpus():
    """Main function to run the corpus generation pipeline."""
    print("Loading raw Tunisian corpus...")
    try:
        raw_dataset = load_dataset(DATASET_NAME, split='train') # Assuming 'train' split contains the data
        print(f"Loaded {len(raw_dataset)} examples from {DATASET_NAME}")
    except Exception as e:
        print(f"Error loading dataset {DATASET_NAME}: {e}")
        return

    processed_data = []
    print("Starting translation and scoring pipeline...")
    for i, example in enumerate(tqdm(raw_dataset, desc="Processing")):
        raw_tn_text = example.get('text', example.get('sentence', None)) # Adjust key based on dataset structure
        if not raw_tn_text:
            continue

        processed_tn = preprocess_sentence(raw_tn_text)
        if not processed_tn or not is_valid_tunisian_arabic(processed_tn):
            continue # Skip invalid or filtered sentences

        best_msa_candidate = None
        best_scores = {}
        final_composite_score = 0.0
        accepted = False
        attempts = 0
        all_candidates = [] # For ensemble agreement

        while attempts < MAX_ATTEMPTS and not accepted:
            attempts += 1
            msa_candidate = translate_tn_to_msa(processed_tn)
            if not msa_candidate:
                continue # Skip if translation failed

            all_candidates.append(msa_candidate)

            # --- Scoring (Replace with actual model calls) ---
            scores = {}
            scores['semantic_similarity'] = calculate_semantic_similarity(processed_tn, msa_candidate) # Pass model if needed
            scores['lm_logprob'] = calculate_lm_fluency(msa_candidate) # Pass model if needed
            # Placeholder for backtranslation - would require another LLM call TN -> MSA -> TN
            scores['backtranslation_score'] = calculate_backtranslation_score(processed_tn, f"[BACKTRANSLATED_{processed_tn}]") # Placeholder
            # Placeholder for ensemble - would require generating multiple candidates
            scores['ensemble_agreement'] = calculate_ensemble_agreement([msa_candidate, f"[CANDIDATE_2_{processed_tn}]", f"[CANDIDATE_3_{processed_tn}]"]) # Placeholder

            composite_score = calculate_composite_score(scores)

            if composite_score > final_composite_score:
               final_composite_score = composite_score
               best_msa_candidate = msa_candidate
               best_scores = scores.copy() # Store the best scores

            if composite_score >= SCORE_THRESHOLD:
               accepted = True
               break # Acceptable candidate found

        # If no acceptable candidate found after MAX_ATTEMPTS, use the best one
        if not accepted and best_msa_candidate:
           accepted = False # Explicitly mark as not accepted if below threshold
            # You might choose to include low-scoring examples with accepted=False
            # For now, let's include the best attempt even if below threshold
            # If you want to discard them, add a condition here.

        # --- Prepare final data record ---
        record = {
            "id": str(uuid.uuid4()),
            "source": best_msa_candidate if best_msa_candidate else "", # MSA as source
            "target": processed_tn, # TN as target
            "source_dialect": raw_tn_text, # Original raw TN text
            "msa_generated": best_msa_candidate if best_msa_candidate else "", # Initial (best) MSA candidate
            "score_composite": final_composite_score,
            "cosine_similarity": best_scores.get('semantic_similarity', 0.0),
            "lm_logprob": best_scores.get('lm_logprob', float('-inf')),
            "backtranslation_score": best_scores.get('backtranslation_score', 0.0),
            "ensemble_agreement": best_scores.get('ensemble_agreement', 0.0),
            "num_attempts": attempts,
            "accepted": accepted,
            "split": "train", # Placeholder, will assign splits later
            "date_generated": datetime.datetime.utcnow().isoformat() + 'Z',
            "model_used": GROQ_MODEL if USE_GROQ else LOCAL_LLM_NAME
        }
        processed_data.append(record)

        # Optional: Save periodically to avoid losing data
        if (i + 1) % 100 == 0:
             print(f"Processed {i+1} examples. Saving checkpoint...")
             # Save checkpoint logic here if needed (e.g., save every 100 examples)

    print(f"Pipeline completed. Total processed examples: {len(processed_data)}")

    # --- Assign Splits ---
    print("Assigning dataset splits...")
    df = pd.DataFrame(processed_data)
    if not df.empty:
        # Shuffle the dataframe
        df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
        total_size = len(df_shuffled)
        train_end = int(SPLIT_RATIO['train'] * total_size)
        val_end = train_end + int(SPLIT_RATIO['validation'] * total_size)

        df_shuffled.loc[:train_end, 'split'] = 'train'
        df_shuffled.loc[train_end:val_end, 'split'] = 'validation'
        df_shuffled.loc[val_end:, 'split'] = 'test'

        # Convert back to list of dicts
        processed_data = df_shuffled.to_dict('records')
    else:
         print("No data to assign splits to.")

    # --- Save to JSONL ---
    print(f"Saving final dataset to {OUTPUT_FILE}...")
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            for record in processed_data:
                f.write(json.dumps(record, ensure_ascii=False) + '\n')
        print("Dataset saved successfully.")
    except Exception as e:
        print(f"Error saving dataset: {e}")

In [ ]:
generate_parallel_corpus()